In [283]:
BASE_DIR = '../'*3
DATASET_DIR = BASE_DIR+'code/util/dataset/dataset_dummy.ipynb'
EXTMODEL_DIR = BASE_DIR + 'code/CNN/cnn_ext_model/ext_model.ipynb'

In [306]:
%run {DATASET_DIR}
%run {EXTMODEL_DIR}

Using device: cpu
Using PyTorch version: 1.9.0


In [307]:
imagenet = DummyDataset('imagenet', 'select',[3,299,299],200)

200


In [308]:
CnnExtModel.set_macro('v3_preproc', 
          ['serial',
              ['conv',{'ksize':3,'stride':2,'chn':32, 'padding':'VALID'}],
              ['conv',{'ksize':3,'chn':32, 'padding':'VALID'}],
              ['conv',{'ksize':3,'chn':64, 'padding':'SAME'}],
              ['max',{'ksize':3, 'stride':2, 'padding':'VALID'}],
              ['conv',{'ksize':1,'chn':80,'padding':'VALID'}],
              ['conv',{'ksize':3,'chn':192,'padding':'VALID'}],
              ['max',{'ksize':3, 'stride':2,'padding':'VALID'}]
          
          ]
                     )

In [309]:
CnnExtModel.set_macro('v3_inception1',
        ['parallel',
            ['conv',{'ksize':1, 'chn':64}],
         
            ['serial',
                ['conv',{'ksize':1,'chn':48}],
                ['conv',{'ksize':5, 'chn':64}]],
         
            ['serial',
                ['conv',{'ksize':1,'chn':64}],
                 ['conv',{'ksize':3,'chn':96}],
                 ['conv',{'ksize':3,'chn':96}]],
         
            ['serial',
                ['avg',{'ksize':3,'stride':1}],
                ['conv',{'ksize':1,'chn':'#chn'}]]
        
        ]
                    
                     )

In [310]:
CnnExtModel.set_macro('v3_resize1',
            ['parallel',
                ['conv',{'ksize':3,'stride':2,'chn':384}],
                ['serial',
                 ['conv',{'ksize':1,'chn':64}],
                 ['conv',{'ksize':3,'chn':94}],
                 ['conv',{'ksize':3,'stride':2,'chn':96}]],
                ['max',{'ksize':3,'stride':2}]
            
            ]                     
                    
                     )

In [311]:
CnnExtModel.set_macro('v3_inception2',
    ['parallel',
        ['conv', {'ksize':1, 'chn':192}],
        ['serial',
            ['conv', {'ksize':[1,1], 'chn':'#chn'}],
            ['conv', {'ksize':[1,7], 'chn':'#chn'}],
            ['conv', {'ksize':[7,1], 'chn':192}]],
        ['serial',
            ['conv', {'ksize':[1,1], 'chn':'#chn'}],
            ['conv', {'ksize':[7,1], 'chn':'#chn'}],
            ['conv', {'ksize':[1,7], 'chn':'#chn'}],
            ['conv', {'ksize':[7,1], 'chn':'#chn'}],
            ['conv', {'ksize':[1,7], 'chn':192}]],
        ['serial',
            ['avg', {'ksize':3, 'stride':1}],
            ['conv', {'ksize':1, 'chn':192}]]])

In [312]:
CnnExtModel.set_macro('v3_resize2',
    ['parallel',
        ['serial',
            ['conv', {'ksize':1, 'chn':192}],
            ['conv', {'ksize':3, 'stride':2, 'chn':320}]],
        ['serial',
            ['conv', {'ksize':[1,1], 'chn':192}],
            ['conv', {'ksize':[1,7], 'chn':192}],
            ['conv', {'ksize':[7,1], 'chn':192}],
            ['conv', {'ksize':[3,3], 'stride':[2,2], 'chn':192}]],
        ['max', {'ksize':3, 'stride':2}]])

In [313]:
CnnExtModel.set_macro('v3_inception3',
    ['parallel',
        ['conv', {'ksize':1, 'chn':320}],
        ['serial',
            ['conv', {'ksize':[3,3], 'chn':384}],
            ['parallel',
                ['conv', {'ksize':[1,3], 'chn':384}],
                ['conv', {'ksize':[3,1], 'chn':384}]]],
        ['serial',
            ['conv', {'ksize':[1,1], 'chn':448}],
            ['conv', {'ksize':[3,3], 'chn':384}],
            ['parallel',
                ['conv', {'ksize':[1,3], 'chn':384}],
                ['conv', {'ksize':[3,1], 'chn':384}]]],
        ['serial',
            ['avg', {'ksize':3, 'stride':1}],
            ['conv', {'ksize':1, 'chn':192}]]])

In [314]:
CnnExtModel.set_macro('v3_postproc',
    ['serial',
        ['avg', {'stride':8}],
        ['dropout', {'keep_prob':0.7}]])

In [315]:
CnnExtModel.set_macro('inception_v3',
    ['serial',
        ['custom', {'name':'v3_preproc'}],
        ['custom', {'name':'v3_inception1', 'args':{'#chn':32}}],
        ['custom', {'name':'v3_inception1', 'args':{'#chn':64}}],
        ['custom', {'name':'v3_inception1', 'args':{'#chn':64}}],
        ['custom', {'name':'v3_resize1'}],
        ['custom', {'name':'v3_inception2', 'args':{'#chn':128}}],
        ['custom', {'name':'v3_inception2', 'args':{'#chn':160}}],
        ['custom', {'name':'v3_inception2', 'args':{'#chn':160}}],
        ['custom', {'name':'v3_inception2', 'args':{'#chn':192}}],
        ['custom', {'name':'v3_resize2'}],
        ['custom', {'name':'v3_inception3'}],
        ['custom', {'name':'v3_inception3'}],
        ['custom', {'name':'v3_postproc'}]])

In [316]:
inception_v3 = CnnExtModel('inception_v3', imagenet,
                   [['custom', {'name':'inception_v3'}]], dump_structure=True)

custom inception_v3
['serial', ['custom', {'name': 'v3_preproc'}], ['custom', {'name': 'v3_inception1', 'args': {'#chn': 32}}], ['custom', {'name': 'v3_inception1', 'args': {'#chn': 64}}], ['custom', {'name': 'v3_inception1', 'args': {'#chn': 64}}], ['custom', {'name': 'v3_resize1'}], ['custom', {'name': 'v3_inception2', 'args': {'#chn': 128}}], ['custom', {'name': 'v3_inception2', 'args': {'#chn': 160}}], ['custom', {'name': 'v3_inception2', 'args': {'#chn': 160}}], ['custom', {'name': 'v3_inception2', 'args': {'#chn': 192}}], ['custom', {'name': 'v3_resize2'}], ['custom', {'name': 'v3_inception3'}], ['custom', {'name': 'v3_inception3'}], ['custom', {'name': 'v3_postproc'}]] macromacromacromacromacro
  serial
    custom v3_preproc
['serial', ['conv', {'ksize': 3, 'stride': 2, 'chn': 32, 'padding': 'VALID'}], ['conv', {'ksize': 3, 'chn': 32, 'padding': 'VALID'}], ['conv', {'ksize': 3, 'chn': 64, 'padding': 'SAME'}], ['max', {'ksize': 3, 'stride': 2, 'padding': 'VALID'}], ['conv', {'ksi

          42: conv, [128, 17, 17]=>[128, 17, 17]pm : 128x128x7x1+128 = 114816
          43: conv, [128, 17, 17]=>[128, 17, 17]pm : 128x128x1x7+128 = 114816
          44: conv, [128, 17, 17]=>[128, 17, 17]pm : 128x128x7x1+128 = 114816
          45: conv, [128, 17, 17]=>[192, 17, 17]pm : 192x128x1x7+192 = 172224
        serial
          46: avg, [768, 17, 17]=>[768, 17, 17]
          47: conv, [768, 17, 17]=>[192, 17, 17]pm : 192x768x1x1+192 = 147648
[Sequential(
  (parallel_0): Conv2d(768, 192, kernel_size=(1, 1), stride=(1, 1), padding=same)
  (parallel_1): Sequential(
    (serial_0): Conv2d(768, 128, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (serial_1): Conv2d(128, 128, kernel_size=(1, 7), stride=(1, 1), padding=same)
    (serial_2): Conv2d(128, 192, kernel_size=(7, 1), stride=(1, 1), padding=same)
  )
  (parallel_2): Sequential(
    (serial_0): Conv2d(768, 128, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (serial_1): Conv2d(128, 128, kernel_size=(7, 1), stride=(1, 

          83: conv, [768, 17, 17]=>[192, 17, 17]pm : 192x768x1x1+192 = 147648
          84: conv, [192, 17, 17]=>[192, 17, 17]pm : 192x192x1x7+192 = 258240
          85: conv, [192, 17, 17]=>[192, 17, 17]pm : 192x192x7x1+192 = 258240
          86: conv, [192, 17, 17]=>[192, 8, 8]pm : 192x192x3x3+192 = 331968
        87: max, [768, 17, 17]=>[768, 8, 8]
[Sequential(
  (parallel_0): Sequential(
    (serial_0): Conv2d(768, 192, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (serial_1): Conv2d(192, 320, kernel_size=(3, 3), stride=(1, 1), padding=same)
  )
  (parallel_1): Sequential(
    (serial_0): Conv2d(768, 192, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (serial_1): Conv2d(192, 192, kernel_size=(1, 7), stride=(1, 1), padding=same)
    (serial_2): Conv2d(192, 192, kernel_size=(7, 1), stride=(1, 1), padding=same)
    (serial_3): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=same)
  )
  (parallel_2): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=same, di

In [270]:
a=inception_v3.modules
# tmplayers=sum(inception_v3.b,[])
len(a)

14

In [271]:
a[13]

Sequential(
  (serial_0): Sequential(
    (serial_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=valid)
    (serial_1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=valid)
    (serial_2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (serial_3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=valid, dilation=1, ceil_mode=False)
    (serial_4): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), padding=valid)
    (serial_5): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), padding=valid)
    (serial_6): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=valid, dilation=1, ceil_mode=False)
  )
  (serial_1): Sequential(
    (parallel_0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (parallel_1): Sequential(
      (serial_0): Conv2d(192, 48, kernel_size=(1, 1), stride=(1, 1), padding=same)
      (serial_1): Conv2d(48, 64, kernel_size=(5, 5), stride=(1, 1), padding=same)
    )
    (parallel_2): Sequential(
    

In [548]:
class Net(nn.Module):
    def __init__(self,layers):
        super(Net, self).__init__()
        self.layer1=nn.Sequential(*layers)
        
    def forward(self, x ):
        
        pass

In [549]:
a=Net(tmplayers)

In [550]:
a

Net(
  (layer1): Sequential(
    (0): Sequential(
      (inception_v3): Sequential(
        (seq): Sequential(
          (seq): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=valid, dilation=1, ceil_mode=False)
        )
      )
    )
    (1): Sequential(
      (parallel): Sequential(
        (seq): Conv2d(192, 32, kernel_size=(1, 1), stride=(1, 1), padding=same)
      )
      (inception_v3): Sequential(
        (seq): Sequential(
          (name): Sequential(
            (seq): Conv2d(192, 32, kernel_size=(1, 1), stride=(1, 1), padding=same)
          )
        )
      )
    )
    (2): Sequential(
      (parallel): Sequential(
        (seq): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), padding=same)
      )
      (inception_v3): Sequential(
        (seq): Sequential(
          (name): Sequential(
            (seq): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), padding=same)
          )
        )
      )
    )
    (3): Sequential(
      (parallel): Sequential(
        

In [ ]:
F_DATA_DIR = BASE_DIR+ 'code/util/dataset/dataset_flowers.ipynb'

%run {F_DATA_DIR}

fd = FlowersDataset([96,96], [96,96,3])

In [ ]:
CnnExtModel.set_macro('flower_preproc',
    ['serial',
        ['conv', {'ksize':3, 'stride':2, 'chn':6, 'actions':'#act'}]])

CnnExtModel.set_macro('flower_inception1',
    ['parallel',
        ['conv', {'ksize':1, 'chn':4, 'actions':'#act'}],
        ['conv', {'ksize':3, 'chn':6, 'actions':'#act'}],
        ['serial',
            ['conv', {'ksize':3, 'chn':6, 'actions':'#act'}],
            ['conv', {'ksize':3, 'chn':6, 'actions':'#act'}]],
        ['serial',
            ['avg', {'ksize':3, 'stride':1}],
            ['conv', {'ksize':1, 'chn':4, 'actions':'#act'}]]])

CnnExtModel.set_macro('flower_resize',
    ['parallel',
        ['conv', {'ksize':3, 'stride':2, 'chn':12, 'actions':'#act'}],
        ['serial',
            ['conv', {'ksize':3, 'chn':12, 'actions':'#act'}],
            ['conv', {'ksize':3, 'stride':2, 'chn':12, 'actions':'#act'}]],
        ['avg', {'ksize':3, 'stride':2}]])

CnnExtModel.set_macro('flower_inception2',
    ['parallel',
        ['conv', {'ksize':1, 'chn':8, 'actions':'#act'}],
        ['serial',
            ['conv', {'ksize':[3,3], 'chn':8, 'actions':'#act'}],
            ['parallel',
                ['conv', {'ksize':[1,3], 'chn':8, 'actions':'#act'}],
                ['conv', {'ksize':[3,1], 'chn':8, 'actions':'#act'}]]],
        ['serial',
            ['conv', {'ksize':[1,1], 'chn':8, 'actions':'#act'}],
            ['conv', {'ksize':[3,3], 'chn':8, 'actions':'#act'}],
            ['parallel',
                ['conv', {'ksize':[1,3], 'chn':8, 'actions':'#act'}],
                ['conv', {'ksize':[3,1], 'chn':8, 'actions':'#act'}]]],
        ['serial',
            ['avg', {'ksize':3, 'stride':1}],
            ['conv', {'ksize':1, 'chn':8, 'actions':'#act'}]]])

CnnExtModel.set_macro('flower_postproc',
    ['serial',
        ['avg', {'stride':6}],
        ['dropout', {'keep_prob':0.7}]])

In [ ]:
CnnExtModel.set_macro('inception_flower',
    ['serial',
        ['custom', {'name':'flower_preproc', 'args':{'#act':'#act'}}],
        ['custom', {'name':'flower_inception1', 'args':{'#act':'#act'}}],
        ['custom', {'name':'flower_resize', 'args':{'#act':'#act'}}],
        ['custom', {'name':'flower_inception1', 'args':{'#act':'#act'}}],
        ['custom', {'name':'flower_resize', 'args':{'#act':'#act'}}],
        ['custom', {'name':'flower_inception2', 'args':{'#act':'#act'}}],
        ['custom', {'name':'flower_resize', 'args':{'#act':'#act'}}],
        ['custom', {'name':'flower_inception2', 'args':{'#act':'#act'}}],
        ['custom', {'name':'flower_postproc', 'args':{'#act':'#act'}}]])

In [ ]:
conf_flower_LA = ['custom', {'name':'inception_flower', 'args':{'#act':'LA'}}]
model_flower_LA = CnnExtModel('model_flower_LA', fd, 
                         conf_flower_LA, dump_structure=True)

In [ ]:
model_flower_LA.exec_all(report=1)